In [ ]:
# Get the data from Cadena et al. 2019
# NOTE: they applied to their images, shown to the monkeys, a circular aperture mask with cosine fade-out
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('../../data/cadena_plosCB19/cadena_ploscb_data.pkl', 'rb') as g:
    loaded_data = pickle.load(g)

# Cadena found that the transfer‐learning approach (using VGG features) reached near-optimal performance with as little as 20% of the available data
SAMPLE_SIZE = 1400
rng = np.random.RandomState(42)
SAMPLE_INDICES = np.random.choice(loaded_data["images"].shape[0], SAMPLE_SIZE, replace=False)

# Check the available keys and shapes
for key, value in loaded_data.items():
    print(f"{key}: {value.shape if isinstance(value, np.ndarray) else type(value)}")

In [ ]:
# Preprocess neural data

responses = loaded_data["responses"]
print("Responses shape:", responses.shape) # (4, 7250, 166) -> 4 repetitions, 7250 images, 166 neurons

# Check how many images have no valid responses across all neurons and repetitions
fully_missing_images = np.isnan(loaded_data["responses"]).all(axis=(0, 2)) # Check over repetitions & neurons
num_fully_missing = fully_missing_images.sum()
print(f"Fully missing images: {num_fully_missing}/{loaded_data['responses'].shape[1]}")

# Load neural responses and average over the 4 repetitions
averaged_responses = np.nanmean(responses, axis=0) # Shape: (7250, 166)
print("Averaged Responses shape:", averaged_responses.shape)

averaged_responses_cleaned = np.nan_to_num(averaged_responses) # Apply Cadena's preprocessing: Replace NaNs with 0
print('Responses sample:', averaged_responses_cleaned[:20])

# Select only the first 3625 samples, because we will only use half the images
processed_neural_responses = averaged_responses_cleaned[SAMPLE_INDICES]
print('Final responses shape:', processed_neural_responses.shape)

In [ ]:
# Prepare images for VGG-19

import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Define VGG-compatible transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.Grayscale(num_output_channels=3), # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply transformation to all images (using half for now due to memory constraints)
sample = loaded_data["images"][SAMPLE_INDICES]
images = np.stack([
    transform(Image.fromarray(img.astype(np.uint8))).numpy() for img in sample
])

print("Transformed Images Shape:", images.shape)  # (7250, 3, 224, 224)

In [ ]:
# extract features in batches and save to disk

# import torch
# import torchvision.models as models
# from torch.utils.data import DataLoader, TensorDataset
# import numpy as np
# import os

# feature_save_path = "vgg_features_batches.npy"

# if os.path.exists(feature_save_path):
#     file_size = os.path.getsize(feature_save_path) / (1024 * 1024)
#     print(f"Features already saved. File Size: {file_size:.2f} MB")
# else:
#     class VGGFeatureExtractor(torch.nn.Module):
#         def __init__(self, layer_idx=8):  # conv3_1 is at index 8 in VGG-19
#             super(VGGFeatureExtractor, self).__init__()
#             self.features = torch.nn.Sequential(*list(models.vgg19(pretrained=True).features.children())[:layer_idx+1])

#         def forward(self, x):
#             return self.features(x)

#     # Device configuration (GPU if available, otherwise CPU)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     feature_extractor = VGGFeatureExtractor().to(device).eval()

#     # Convert images to Torch tensor dataset
#     images_tensor = torch.tensor(images, dtype=torch.float32).to(device)  # (7250, 3, 224, 224)

#     # DataLoader with smaller batch size
#     batch_size = 8  # Reduce if still running out of memory
#     dataset = TensorDataset(images_tensor)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

#     # Process images in batches and save incrementally
#     with torch.no_grad():  # Disable autograd for memory efficiency
#         for batch_idx, (batch,) in enumerate(dataloader):
#             batch = batch.to(device)  # Move batch to GPU (if available)
#             features = feature_extractor(batch).cpu().numpy()  # Move to CPU immediately
            
#             # Append to .npy file (save in small increments)
#             with open(feature_save_path, "ab") as f:  # "ab" = append binary mode
#                 np.save(f, features)
            
#             # Free memory
#             del batch, features
#             torch.cuda.empty_cache()  # Clears GPU cache

#             print(f"Processed batch {batch_idx + 1}/{len(dataloader)} and saved to disk")

#     print(f"Feature extraction complete! Saved to {feature_save_path}")

In [ ]:
# memory-efficient writing features to disk

# import torch
# import torchvision.models as models
# import numpy as np
# import os
# from torch.utils.data import DataLoader, TensorDataset

# # Define feature extractor (VGG-19 up to conv3_1)
# class VGGFeatureExtractor(torch.nn.Module):
#     def __init__(self, layer_idx=8):  # Extract conv3_1 features
#         super(VGGFeatureExtractor, self).__init__()
#         self.features = torch.nn.Sequential(*list(models.vgg19(pretrained=True).features.children())[:layer_idx+1])

#     def forward(self, x):
#         return self.features(x)

# # Device configuration (Use GPU if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# feature_extractor = VGGFeatureExtractor().to(device).eval()

# # Convert images to Torch tensor dataset
# images_tensor = torch.tensor(images, dtype=torch.float32).to(device)  # (7250, 3, 224, 224)

# # DataLoader for batch processing
# batch_size = 8  # Adjust based on available memory
# dataset = TensorDataset(images_tensor)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# # File to save extracted features
# feature_save_path = "vgg_features_batches.npz"

# # Remove old file if it exists
# if os.path.exists(feature_save_path):
#     os.remove(feature_save_path)

# # List to store batch paths
# batch_files = []

# # Extract and save features batch-wise
# with torch.no_grad():  # Disable autograd for efficiency
#     for batch_idx, (batch,) in enumerate(dataloader):
#         batch = batch.to(device)  # Move batch to GPU (if available)
#         features = feature_extractor(batch).cpu().numpy()  # Move to CPU

#         # Save each batch as a separate .npy file
#         batch_filename = f"vgg_features_batch_{batch_idx}.npy"
#         np.save(batch_filename, features)
#         batch_files.append(batch_filename)  # Keep track of saved files

#         # Free memory
#         del batch, features
#         torch.cuda.empty_cache()

#         print(f"Processed batch {batch_idx + 1}/{len(dataloader)} - Written to {batch_filename}")

# # **Merge All Batches into One Final .npz File**
# print("Merging batches into final compressed file...")
# all_features = [np.load(f) for f in batch_files]  # Load each batch one at a time
# final_features = np.concatenate(all_features, axis=0)  # Merge efficiently
# np.savez_compressed(feature_save_path, features=final_features)  # Save final file

# # Delete temporary batch files to save space
# for file in batch_files:
#     os.remove(file)

# print(f"Feature extraction complete! Final dataset saved to {feature_save_path}")

In [ ]:
import torch
import torchvision.models as models
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os

PATH_TO_FEATURES = "../../data/cadena_plosCB19/vgg_features.npy"

if os.path.exists(PATH_TO_FEATURES):
    print(f"Features file '{PATH_TO_FEATURES}' exists. Loading features...")
else:
    print(f"Features file does not exist. Extracting features...")
    
    # Define feature extractor (VGG-19 up to conv3_1)
    class VGGFeatureExtractor(torch.nn.Module):
        def __init__(self):
            super(VGGFeatureExtractor, self).__init__()
            # Extract feature maps (256, 56, 56) from conv3_1 layer (BEFORE max pooling)
            self.features = torch.nn.Sequential(*list(models.vgg19(pretrained=True).features.children())[:14])

        def forward(self, x):
            return self.features(x)

    # Device configuration (Use GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    feature_extractor = VGGFeatureExtractor().to(device).eval()

    # Convert images to Torch tensor dataset
    images_tensor = torch.tensor(images, dtype=torch.float32).to(device)  # Shape: (3625, 3, 224, 224)

    # DataLoader for batch processing
    batch_size = 8
    dataset = TensorDataset(images_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Store all batches in memory
    all_batches = []

    # Extract features batch-wise and store in RAM
    with torch.no_grad():  # Disable autograd for efficiency
        for batch_idx, (batch,) in enumerate(dataloader):
            batch = batch.to(device)  # Move batch to GPU (if available)
            features = feature_extractor(batch).cpu().numpy()  # Move to CPU

            all_batches.append(features)  # Store batch in memory

            # Free memory
            del batch, features
            torch.cuda.empty_cache()

            print(f"Processed batch {batch_idx + 1}/{len(dataloader)}")

    # Concatenate all batches into a single NumPy array
    vgg_features = np.concatenate(all_batches, axis=0)
    print("Final Features Shape:", vgg_features.shape) # Expected: (3625, 256, 56, 56)
    np.save(PATH_TO_FEATURES, vgg_features)

In [ ]:
# Ridge regression

import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

vgg_features = np.load(PATH_TO_FEATURES, mmap_mode="r") # Use mmap_mode for large files
X = vgg_features.reshape(vgg_features.shape[0], -1) # Flatten from (3625, 256, 56, 56) (3625, feature_dimensions)
y = processed_neural_responses
print("Final Shapes -> X:", X.shape, "y:", processed_neural_responses.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ridge_reg = Ridge(alpha=10000000)
ridge_reg.fit(X_train, y_train)
y_pred = ridge_reg.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"Test R² Score: {r2:.3f}")

def compute_fev(y_true, y_pred):
    noise_variance = np.var(y_true - np.mean(y_true, axis=0), axis=0)
    total_variance = np.var(y_true, axis=0)
    explained_variance = np.var(y_pred, axis=0)
    fev = 1 - (total_variance - explained_variance) / (total_variance - noise_variance)
    return np.mean(fev)

fev_score = compute_fev(y_test, y_pred)
print(f"Fraction of Explainable Variance (FEV): {fev_score:.3f}")

In [ ]:
# Fit a Poisson regression model

import numpy as np
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

vgg_features = np.load(PATH_TO_FEATURES, mmap_mode="r") # Use mmap_mode for large files

# X: (3625, num_features) and y: (3625, 166)
X = vgg_features.reshape(vgg_features.shape[0], -1) # Flatten from (3625, 256, 56, 56) (3625, feature_dimensions)
y = processed_neural_responses
print("Final Shapes -> X:", X.shape, "y:", processed_neural_responses.shape)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_neurons = y_train.shape[1]
models = []
y_pred_test_all = np.zeros_like(y_test)
r2_list = []

# Loop over each neuron (output dimension)
for i in range(n_neurons):
    # Fit a separate PoissonRegressor for neuron i
    model = PoissonRegressor(alpha=0.01, max_iter=1000)
    model.fit(X_train, y_train[:, i])
    models.append(model)
    
    # Predict on the test set for this neuron
    y_pred = model.predict(X_test)
    y_pred_test_all[:, i] = y_pred
    
    # Compute R² for this neuron and store it
    r2 = r2_score(y_test[:, i], y_pred)
    r2_list.append(r2)

# Compute mean R² across neurons
mean_r2 = np.mean(r2_list)
print(f"Mean Test R² Score: {mean_r2:.3f}")

# Optionally, define the FEV function and compute average FEV
def compute_fev(y_true, y_pred):
    # Compute noise variance, total variance, and explained variance for each neuron
    noise_variance = np.var(y_true - np.mean(y_true, axis=0), axis=0)
    total_variance = np.var(y_true, axis=0)
    explained_variance = np.var(y_pred, axis=0)
    fev = 1 - (total_variance - explained_variance) / (total_variance - noise_variance)
    return np.mean(fev)

fev_score = compute_fev(y_test, y_pred_test_all)
print(f"Mean Fraction of Explainable Variance (FEV): {fev_score:.3f}")